In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
sns.set()

%matplotlib inline
%config InlineBackend.figure_format="retina"

In [2]:
np.set_printoptions(suppress=True)

Based on *An Introduction to Counterfactual Regret Minimization* (Neller, Lanctot)

- `0`: no reward
- `1`: positive reward
- `-1`: negative reward

In [3]:
# Actions
ROCK, PAPER, SCISSORS = 0, 1, 2
num_actions = 3

# Stategy
opp_strategy = [0.4, 0.3, 0.3]

# Normal form (Payoff)
payoff = np.array([
    [[ 0,  0], [-1,  0], [ 1, -1]],
    [[ 1, -1], [ 0,  0], [-1,  1]],
    [[-1,  1], [ 1, -1], [ 0,  0]],
], dtype=float)

def value(x, y):
    return payoff[x, y, 0]

In [4]:
# Examples
print("Rock vs Rock    :", value(ROCK, ROCK))
print("Paper vs Rock   :", value(PAPER, ROCK))
print("Scissors vs Rock:", value(SCISSORS, ROCK))

Rock vs Rock    : 0.0
Paper vs Rock   : 1.0
Scissors vs Rock: -1.0


In [20]:
def normalize(strategy):
    normalizing_sum = np.sum(strategy)
    if normalizing_sum > 0:
        return strategy / normalizing_sum
    else:
        return np.ones(strategy.shape[0]) / strategy.shape[0]

def get_strategy(regret_sum):
    # remove negative regrets
    return normalize(np.maximum(regret_sum, 0))

def avg_strategy(strategy_sum):
    return normalize(strategy_sum)
    #x = (regret_sum - np.min(regret_sum)) / (np.max(regret_sum) - np.min(regret_sum))
    #return x / sum(x)

def get_action(strategy):
    strategy = strategy / np.sum(strategy)
    return np.searchsorted(np.cumsum(strategy), random.random())

In [21]:
# Training one player
opp_strategy   = [0.4, 0.3, 0.3]
regret_sum     = np.zeros(num_actions, dtype=float)
strategy_sum   = np.zeros(num_actions, dtype=float)
random.seed(1)
for _ in range(100_000):
    # Get regret-matched mixed-strategy actions
    strategy = get_strategy(regret_sum)
    strategy_sum += strategy
    
    # Play game
    my_action  = get_action(strategy)
    opp_action = get_action(opp_strategy)
    
    # Compute action utilities (for rock paper scissors)
    action_utility = payoff[:, opp_action, 0]
    
    # Accumulate action regrets
    regret_sum += action_utility - action_utility[my_action]

print("Best strategy:", avg_strategy(strategy_sum))

Best strategy: [0.00008983 0.99957457 0.00033561]


### Exercise: RPS Equilibrium

In [27]:
# Training two players 
regret_sum_p1   = np.zeros(num_actions)
regret_sum_p2   = np.zeros(num_actions)
strategy_sum_p1 = np.zeros(num_actions)
strategy_sum_p2 = np.zeros(num_actions)

random.seed(None)
for _ in range(100_000):
    # Get regret-matched mixed strategy actions
    strategy_p1 = get_strategy(regret_sum_p1)
    strategy_p2 = get_strategy(regret_sum_p2)
    strategy_sum_p1 += strategy_p1
    strategy_sum_p2 += strategy_p2
    
    # Play game according to strategy
    action_p1 = get_action(strategy_p1)
    action_p2 = get_action(strategy_p2)
    
    # Compute action utility
    action_utility_p1 = payoff[:, action_p1, 0]
    action_utility_p2 = payoff[action_p2, :, 0]
    regret_sum_p1 += action_utility_p1 - action_utility_p1[action_p1]
    regret_sum_p2 += action_utility_p2 - action_utility_p2[action_p2]
print("Best strategy for P1: ", np.round(avg_strategy(strategy_sum_p1), 3))
print("Best strategy for P1: ", np.round(avg_strategy(strategy_sum_p2), 3))

Best strategy for P1:  [0.335 0.333 0.332]
Best strategy for P1:  [0.331 0.334 0.334]


--> converges to *Nash Equilibrium*